In [25]:
from collections import defaultdict
import numpy as np
import myfm
from myfm import RelationBlock
from scipy import sparse as sps

from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

def load_cil(dataset="split"):

    file = "data_train" if dataset != "test" else "sampleSubmission"
    data = pd.read_csv(f'../../delivery/data/{file}.csv', index_col=0)
    data['user'] = data.index.str.split('_').str[0].str[1:].astype('int32')
    data['movie'] = data.index.str.split('_').str[1].str[1:].astype('int32')
    data.rename(columns={'Prediction': 'rating'}, inplace=True)
    data['rating'] = data['rating'].astype('uint8')
    data = data[['user', 'movie', 'rating']]

    data['user'] = data['user']
    data['movie'] = data['movie']
    # print("Subtracted {} from user and movie".format(1))

    user_num = 10000  # int(data['user'].max() + 1)
    movie_num = 1000  # int(data['movie'].max() + 1)
    print("User num: {}, Movie num: {}".format(user_num, movie_num))

    train_data = val_data = None
    if dataset == "test":
        val_data = data
    elif dataset == "train":
        train_data = data
    else:
        train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

    return train_data, val_data, user_num, movie_num

In [26]:
train_data, val_data, user_num, movie_num = load_cil("train")

User num: 10000, Movie num: 1000


In [27]:
_, val_data, user_num, movie_num = load_cil("test")

User num: 10000, Movie num: 1000


In [28]:
# index "0" is reserved for unknown ids.
user_to_index = defaultdict(lambda : 0, { uid: i+1 for i,uid in enumerate(np.unique(train_data.user)) })
movie_to_index = defaultdict(lambda: 0, { mid: i+1 for i,mid in enumerate(np.unique(train_data.movie))})
USER_ID_SIZE = len(user_to_index) + 1
MOVIE_ID_SIZE = len(movie_to_index) + 1

In [29]:
# The flags to control the included features.
use_iu = True # use implicit user feature
use_ii = True # use implicit item feature

movie_vs_watched = dict()
user_vs_watched = dict()
for row in train_data.itertuples():
    user_id = row.user
    movie_id = row.movie
    movie_vs_watched.setdefault(movie_id, list()).append(user_id)
    user_vs_watched.setdefault(user_id, list()).append(movie_id)
X_date_train, X_date_test = (None, None)

In [30]:
# given user/movie ids, add additional infos and return it as sparse
def augment_user_id(user_ids):
    Xs = []
    X_uid = sps.lil_matrix((len(user_ids), USER_ID_SIZE))
    for index, user_id in enumerate(user_ids):
        X_uid[index, user_to_index[user_id]] = 1
    Xs.append(X_uid)
    if use_iu:
        X_iu = sps.lil_matrix((len(user_ids), MOVIE_ID_SIZE))
        for index, user_id in enumerate(user_ids):
            watched_movies = user_vs_watched.get(user_id, [])
            normalizer = 1 / max(len(watched_movies), 1) ** 0.5
            for uid in watched_movies:
                X_iu[index, movie_to_index[uid]] = normalizer
        Xs.append(X_iu)
    return sps.hstack(Xs, format='csr')

def augment_movie_id(movie_ids):
    Xs = []
    X_movie = sps.lil_matrix((len(movie_ids), MOVIE_ID_SIZE))
    for index, movie_id in enumerate(movie_ids):
        X_movie[index, movie_to_index[movie_id]] = 1
    Xs.append(X_movie)

    if use_ii:
        X_ii = sps.lil_matrix((len(movie_ids), USER_ID_SIZE))
        for index, movie_id in enumerate(movie_ids):
            watched_users = movie_vs_watched.get(movie_id, [])
            normalizer = 1 / max(len(watched_users), 1) ** 0.5
            for uid in watched_users:
                X_ii[index, user_to_index[uid]] = normalizer
        Xs.append(X_ii)


    return sps.hstack(Xs, format='csr')

In [31]:
train_uid_unique, train_uid_index = np.unique(train_data.user, return_inverse=True)
train_mid_unique, train_mid_index = np.unique(train_data.movie, return_inverse=True)
user_data_train = augment_user_id(train_uid_unique)
movie_data_train = augment_movie_id(train_mid_unique)

test_uid_unique, test_uid_index = np.unique(val_data.user, return_inverse=True)
test_mid_unique, test_mid_index = np.unique(val_data.movie, return_inverse=True)
user_data_test = augment_user_id(test_uid_unique)
movie_data_test = augment_movie_id(test_mid_unique)

In [32]:
block_user_train = RelationBlock(train_uid_index, user_data_train)
block_movie_train = RelationBlock(train_mid_index, movie_data_train)
block_user_test = RelationBlock(test_uid_index, user_data_test)
block_movie_test = RelationBlock(test_mid_index, movie_data_test)

In [33]:
y_train = train_data.rating.values
y_test = val_data.rating.values

In [34]:
FM_RANK = 11
N_ITER = 200 # 512
N_KEPT_SAMPLES = 200 # 200

In [35]:
fm_rb = myfm.MyFMRegressor(rank=FM_RANK).fit(
    X_date_train, y_train,
    X_rel=[block_user_train, block_movie_train],
    group_shapes=[USER_ID_SIZE, MOVIE_ID_SIZE, MOVIE_ID_SIZE, USER_ID_SIZE],
    n_iter=N_ITER, n_kept_samples=N_KEPT_SAMPLES
)

  0%|          | 0/200 [00:00<?, ?it/s]

alpha = 1.14 w0 = 3.65 : 100%|██████████| 200/200 [01:43<00:00,  1.92it/s]


In [36]:
test_prediction = fm_rb.predict(
    X_date_test,
    X_rel=[block_user_test, block_movie_test]
)

In [37]:
test_prediction = np.clip(test_prediction, 1, 5)
val_data['Prediction'] = test_prediction

In [38]:
rmse = ((y_test - val_data['Prediction']) ** 2).mean() ** 0.5
mae = np.abs(y_test - val_data['Prediction']).mean()
print(f'rmse={rmse}, mae={mae}')

rmse=1.0174302811203404, mae=0.8952395290973649


In [39]:
# 512, 50 ->  rmse=0.9774482247691249
# 512, 10 -> rmse=0.975...

In [40]:
val_data['Prediction'].to_csv(f"../../delivery/outputs/BFM/predictions_bfm_plus_plus_grouped_test_finetuned.csv")

# Hyperparameter tuning

In [43]:
N_ITER = 512
N_KEPT_SAMPLES = 200

fm_rank_list = np.arange(5,15)
rmse_list = []

best_fm_rank = None
best_rmse = 100
best_mae = 100
best_model = None

for FM_RANK in fm_rank_list:
    N_KEPT_SAMPLES = min(200, N_ITER)
    fm_rb = myfm.MyFMRegressor(rank=FM_RANK).fit(
        X_date_train, y_train,
        X_rel=[block_user_train, block_movie_train],
        group_shapes=[USER_ID_SIZE, MOVIE_ID_SIZE, MOVIE_ID_SIZE, USER_ID_SIZE],
        n_iter=N_ITER, n_kept_samples=N_KEPT_SAMPLES
    )
    test_prediction = fm_rb.predict(
        X_date_test,
        X_rel=[block_user_test, block_movie_test]
    )
    test_prediction[test_prediction > 5] = 5
    test_prediction[test_prediction < 1] = 1
    rmse = ((y_test - test_prediction) ** 2).mean() ** 0.5
    mae = np.abs(y_test - test_prediction).mean()
    print(f'rank={FM_RANK}, rmse={rmse}, mae={mae}')
    rmse_list.append(rmse)
    if rmse < best_rmse:
        best_rmse = rmse
        best_mae = mae
        best_fm_rank = FM_RANK
        best_model = fm_rb
        val_data['Prediction'] = test_prediction
        val_data['Prediction'].to_csv(f"../../delivery/outputs/BFM/predictions_bfm_plus_plus_grouped_tuned.csv")

# plot rmse vs fm_rank using seaborn
sns.set_theme(style="whitegrid")
ax = sns.lineplot(x=fm_rank_list, y=rmse_list)
ax.set(xlabel='fm_rank', ylabel='rmse')
plt.show()

  0%|          | 0/512 [00:00<?, ?it/s]

alpha = 1.09 w0 = 3.57 :   9%|▉         | 45/512 [00:14<02:33,  3.03it/s]


KeyboardInterrupt: 

# Investigate std of predictions

In [ ]:
# get all predictions
# w0_samples is of shape (200,)
# w_samples is of shape (200, 22004)
# V_samples is of shape (200, 22004, 10)

# get single prediction
def make_prediction(fm_rb, input):

    w0 = fm_rb.w0_samples
    w1 = np.sum(fm_rb.w_samples * input, axis=1)

    w2 = np.zeros_like(w0)
    nonzero_indices = input.nonzero()[0]
    nonzero_values = input[nonzero_indices]
    v_relevant = fm_rb.V_samples[:,nonzero_indices]
    for p in range(v_relevant.shape[1] - 1):
        for q in range(p+1, v_relevant.shape[1]):
            w2 += np.sum(v_relevant[:,p] * v_relevant[:,q], axis=1) * nonzero_values[p] * nonzero_values[q]

    print(np.mean(w0), np.mean(w1), np.mean(w2))
    print(np.std(w0), np.std(w1), np.std(w2))

    predictions = w0 + w1 + w2

    prediction = np.mean(predictions)
    std = np.std(predictions)
    return prediction, std

In [ ]:
val_data['user'][0], val_data['movie'][0], val_data['rating'][0], val_data['Prediction'][0]

KeyError: 'Prediction'

In [ ]:
item = 0

user_data_item = user_data_test[test_uid_index[item]].toarray().reshape(-1)
movie_data_item = movie_data_test[test_mid_index[item]].toarray().reshape(-1)

input = np.concatenate((user_data_item, movie_data_item))
make_prediction(fm_rb, input)

3.7951400466269263 0.008322847102688925 -0.023780861519833524
0.03888429353338309 0.12963034520524044 0.05187523308357968


(3.7796820322097817, 0.13742346209977063)

# Investigate std of predictions in matrix form

In [ ]:
from scipy.sparse import hstack
user_data_items = user_data_test[test_uid_index]
movie_data_items = movie_data_test[test_mid_index]
input = hstack([user_data_items, movie_data_items])
nr_items = 2000 # input.shape[0]
input = input[:nr_items]

In [ ]:
nr_items

235391

In [ ]:
from tqdm import tqdm
from scipy import sparse

predictions = []
for sample in tqdm(range(1)):
    w0 = fm_rb.w0_samples[sample] * np.ones(nr_items)
    w1 = fm_rb.w_samples[sample] @ input.T

    V = fm_rb.V_samples[sample]
    dot_product = V @ V.T

    # sparse section
    dot_product = sparse.triu(dot_product, k=1, format='csr')
    w2 = (input @ dot_product).multiply(input)
    w2 = w2.sum(axis=1)
    w2 = np.array(w2.tolist()).flatten()

    prediction = w0 + w1 + w2
    predictions.append(prediction)

100%|██████████| 1/1 [00:35<00:00, 35.73s/it]


In [ ]:
np.mean(predictions, axis=0)

array([3.63212013, 3.82491669, 3.97428969, 4.132301  , 3.94141752])

In [ ]:
test_prediction[:5]

array([3.56289154, 3.94434783, 4.05418888, 3.86982994, 3.75007395])